<font size="5">Ingest WDI - "GDP per capita" data into Trino pipeline</font>

In [1]:
import os
import pathlib
import json
import osc_ingest_trino as osc
import trino
from sqlalchemy.engine import create_engine

import pandas as pd
from openscm_units import unit_registry
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType

from dotenv import dotenv_values, load_dotenv
import pathlib
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Load Environment Variables

In [2]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_wdi_gdp'

In [4]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [5]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [8]:
# Show available schemas to ensure trino connection is set correctly
#schema_read = connection.execute(f'show schemas from {ingest_catalog}')
schema_read = connection.execute(f'select schema_name from {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

ObjectNotExecutableError: Not an executable object: 'show schemas from osc_datacommons_dev'

Load GDP file (updated sporadically from https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD)

In [8]:
import pandas as pd
import ParseXLS as parser


## GDP
ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/WDI/API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv')
ticker_file.download_file(f'/tmp/API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv')



df = parser.process('WDI_GDP.ini','WDI_GDP.csv') 



## GDP PPP

ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/WDI/API_NY.GDP.MKTP.PP.CD_DS2_en_csv_v2.csv')
ticker_file.download_file(f'/tmp/API_NY.GDP.MKTP.PP.CD_DS2_en_csv_v2.csv')



df2 = parser.process('WDI_GDP_PPP.ini','WDI_GDP_PPP.csv') 

# combine both dataframes

df =pd.concat([df,df2])

df=df.astype({'validity_date': 'int32'})
df= df.convert_dtypes()
df.info(verbose=True)
df= df[['rec_source','data_provider','country_iso_code','country_name','validity_date','attribute','value','value_units']].dropna(subset=['value'])
#df = df.convert_dtypes()
#print(df.info(verbose=True))
#df
#df


WDI_GDP.ini
file_list:
['/tmp/API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv']
/tmp/API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv
2
csv
/tmp/API_NY.GDP.MKTP.CD_DS2_en_csv_v
WDI_GDP_PPP.ini
file_list:
['/tmp/API_NY.GDP.MKTP.PP.CD_DS2_en_csv_v2.csv']
/tmp/API_NY.GDP.MKTP.PP.CD_DS2_en_csv_v2.csv
2
csv
/tmp/API_NY.GDP.MKTP.PP.CD_DS2_en_csv_v


NameError: name 'XXXXX' is not defined

In [ ]:
#%run -i TransposeXLS.py --config WDI.ini --output=WDI.csv 

In [ ]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df) 


In [ ]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['validity_date']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
#print(qres.fetchall())

In [ ]:
# Delete the data for the related attribute 
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table} 
"""
qres = engine.execute(sql)
print(qres.fetchall())

In [ ]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


In [ ]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
df.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

In [ ]:
import pandas as pd
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}" + " where country_iso_code='BHS' order by validity_date desc"""
pd.read_sql(sql, engine)
